# Step-Back Prompting (Question-Answering)

One prompting technique called "Step-Back" prompting can improve performance on complex questions by first asking a "step back" question. This can be combined with regular question-answering applications by then doing retrieval on both the original and step-back question.

Read the paper [here](https://arxiv.org/abs/2310.06117)

See an excellent blog post on this by Cobus Greyling [here](https://cobusgreyling.medium.com/a-new-prompt-engineering-technique-has-been-introduced-called-step-back-prompting-b00e8954cacb)

In this cookbook we will replicate this technique. We modify the prompts used slightly to work better with chat models.

In [1]:
from dotenv import load_dotenv
load_dotenv
import rich

In [36]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

In [3]:
# Few Shot Examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [5]:
question_gen = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0) | StrOutputParser()

In [6]:
question = "was chatgpt around while trump was president?"
question_gen.invoke({"question": question})

'when was ChatGPT developed and released?'

In [8]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

search = DuckDuckGoSearchAPIWrapper(max_results=4)


def retriever(query):
    return search.run(query)

In [9]:
retriever(question)

'Republican presidential nominee former President Donald Trump speaks at a campaign rally in Allentown, Pa., Tuesday, Oct. 29, 2024. (AP Photo/Matt Rourke) Donald Trump talks to the media in 2019 in Washington D.C. As the president-elect prepares to take office, speculation surrounds potential conflicts within his newly formed Cabinet. Republican presidential nominee, former President Donald Trump dances off stage at the conclusion of a campaign rally at the J.S. Dorton Arena on November 04, 2024 in Raleigh, North Carolina. President-elect Donald Trump is poised to be at the helm as AI infrastructure develops, says OpenAI CFO Sarah Friar. Despite tensions with Elon Musk, OpenAI thrives, showcasing innovative tools like Sora. Friar anticipates AI advances in 2024, collaborating with Microsoft while managing growth challenges.'

In [10]:
retriever(question_gen.invoke({"question": question}))

"April 23, 2023 - OpenAI released ChatGPT plugins, GPT-3.5 with browsing, and GPT-4 with browsing in ALPHA. ... or will the outcome of those cases slow the development of future training models? Unlike previous AI models, ChatGPT was developed in a way that keeps up with follow up questions and giving answers that felt personal and relevant. ... In 2023, OpenAI released GPT-4 to ChatGPT Plus subscribers for a new level of detail and precision to the model's responses. This update allowed ChatGPT to handle more complex questions and ... June 2020: GPT-3 was released, constituting a major leap in AI technology with its 175 billion parameters. It was employed to perform various tasks, such as drafting emails, writing articles, composing poetry, and generating code, showcasing its versatile applications. November 30, 2022: ChatGPT was launched using GPT-3.5, and it quickly went ... OpenAI was founded in 2015 and since then has developed and introduced ChatGPT to the public, led by CEO Sam 

In [193]:
# response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

# {normal_context}
# {step_back_context}

# Original Question: {question}
# Answer:"""
# response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

In [33]:
from langchain import hub

response_prompt = hub.pull("langchain-ai/stepback-answer")

In [37]:
template = """You are an expert of world knowledge. 
I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. 
Otherwise, ignore them if they are not relevant.

<normal_context>
# {normal_context}
</normal_context>

<step_back_context>
# {step_back_context}
</step_back_context>


# Original Question: {question}
# Answer:"""

response_prompt = PromptTemplate.from_template(template)

In [34]:
rich.print(response_prompt)

PromptTemplate(
    input_variables=['normal_context', 'question', 'step_back_context'],
    input_types={},
    partial_variables={},
    metadata={
        'lc_hub_owner': 'langchain-ai',
        'lc_hub_repo': 'stepback-answer',
        'lc_hub_commit_hash': 'e9860e84090dcfa49a71be16483370a599223ed06be88597cc802d9c9201e914'
    },
    template='You are an expert of world knowledge. I am going to ask you a question. Your response should be 
comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they 
are not relevant.\n\n{normal_context}\n{step_back_context}\n\nOriginal Question: {question}\nAnswer:'
)

In [28]:
chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,

        # Retrieve context using the step-back question
        "step_back_context": question_gen,
        # "step_back_context": RunnableLambda(lambda x: x["question"])| question_gen | retriever,

        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)
    | StrOutputParser()
)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
test_chain = (
    # {
    #     # Retrieve context using the normal question
    #     "normal_context": RunnableLambda(lambda x: x["question"]),

    #     # Retrieve context using the step-back question
    #     # "step_back_context": question_gen | retriever,
    #     "step_back_context": RunnableLambda(lambda x: x["question"])| question_gen,

    #     # Pass on the question
    #     "question": lambda x: x["question"],
    # }
# =====================================================================
    {'normal_context': RunnablePassthrough(),
     'step_back_context': RunnablePassthrough() | question_gen,
     'question': RunnablePassthrough()
     }

    | response_prompt
)
test_chain.invoke({"question": question})
rich.print(test_chain)

RunnableSequence(
    first=RunnableParallel(
        steps__={
            'normal_context': RunnablePassthrough(),
            'step_back_context': RunnableSequence(
                first=RunnablePassthrough(),
                middle=[
                    ChatPromptTemplate(
                        input_variables=['question'],
                        input_types={},
                        partial_variables={},
                        messages=[
                            SystemMessagePromptTemplate(
                                prompt=PromptTemplate(
                                    input_variables=[],
                                    input_types={},
                                    partial_variables={},
                                    template='You are an expert at world knowledge. Your task is to step back and 
paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:'
                                ),
                                additional_kwargs={}
                            ),
                            FewShotChatMessagePromptTemplate(
                                examples=[
                                    {
                                        'input': 'Could the members of The Police perform lawful arrests?',
                                        'output': 'what can the members of The Police do?'
                                    },
                                    {
                                        'input': 'Jan Sindel’s was born in what country?',
                                        'output': 'what is Jan Sindel’s personal history?'
                                    }
                                ],
                                input_variables=[],
                                input_types={},
                                partial_variables={},
                                example_prompt=ChatPromptTemplate(
                                    input_variables=['input', 'output'],
                                    input_types={},
                                    partial_variables={},
                                    messages=[
                                        HumanMessagePromptTemplate(
                                            prompt=PromptTemplate(
                                                input_variables=['input'],
                                                input_types={},
                                                partial_variables={},
                                                template='{input}'
                                            ),
                                            additional_kwargs={}
                                        ),
                                        AIMessagePromptTemplate(
                                            prompt=PromptTemplate(
                                                input_variables=['output'],
                                                input_types={},
                                                partial_variables={},
                                                template='{output}'
                                            ),
                                            additional_kwargs={}
                                        )
                                    ]
                                )
                            ),
                            HumanMessagePromptTemplate(
                                prompt=PromptTemplate(
                                    input_variables=['question'],
                                    input_types={},
                                    partial_variables={},
                                    template='{question}'
                                ),
                                additional_kwargs={}
                            )
                        ]
                    ),
                    ChatOpenAI(
                    

In [21]:
chain.invoke({"question": question})

"ChatGPT was not available during Donald Trump's presidency, which lasted from January 20, 2017, to January 20, 2021. The model was developed by OpenAI and was first released in November 2022, after Trump's term had ended. Therefore, it did not exist while he was in office."

## Baseline

In [17]:
response_prompt_template = """You are an expert of world knowledge. I am going to ask you a question. Your response should be comprehensive and not contradicted with the following context if they are relevant. Otherwise, ignore them if they are not relevant.

{normal_context}

Original Question: {question}
Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

In [18]:
chain = (
    {
        # Retrieve context using the normal question (only the first 3 results)
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)
    | StrOutputParser()
)

In [19]:
chain.invoke({"question": question})

"ChatGPT, developed by OpenAI, was released in November 2022, which means it was not available during Donald Trump's presidency, which lasted from January 20, 2017, to January 20, 2021. Therefore, ChatGPT did not exist while Trump was in office."